In [1]:
import os
from typing import Tuple
from tensorflow.python.keras.utils.generic_utils import Progbar
from tf_agents.environments import suite_gym, parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.replay_buffers import tf_uniform_replay_buffer, episodic_replay_buffer
from tf_agents.trajectories.trajectory import Trajectory
from tf_agents.networks import actor_distribution_network
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
from tf_agents.policies import random_tf_policy
import tensorflow as tf
import numpy as np
from tf_agents.utils import common
from reinforcement_learning.sac_training import NumberOfSafetyViolations
from tf_agents.trajectories import time_step as ts, policy_step, trajectory
from reinforcement_learning import sac_training
from reinforcement_learning import labeling_functions

In [2]:
py_env = suite_gym.load('Pendulum-v0')
py_env.reset()
tf_env = tf_py_environment.TFPyEnvironment(py_env)
tf_env.time_step_spec()


TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(3,), dtype=tf.float32, name='observation', minimum=array([-1., -1., -8.], dtype=float32), maximum=array([1., 1., 8.], dtype=float32)))

In [4]:
def display_safe_labeling(trajectory):
    label = labeling_functions['Pendulum-v0'](trajectory.observation)
    if tf.reduce_any(label):
        print(label)

tf_env = tf_py_environment.TFPyEnvironment(py_env)
policy = random_tf_policy.RandomTFPolicy(time_step_spec=tf_env.time_step_spec(), action_spec=tf_env.action_spec())
dynamic_episode_driver.DynamicEpisodeDriver(tf_env, policy, num_episodes=15,
                                            observers=[display_safe_labeling,
                                                       lambda _: py_env.render(mode='human')]).run()


tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor([ True False False], shape=(3,), dtype=bool)
tf.Tensor(

KeyboardInterrupt: 